In [1]:
# %pip install pdfminer.six --user
# %pip install PyMuPDF --user
# %pip install pillow --user
# %pip install pdfplumber --user
# %pip install tabula-py
# %pip install PyPDF2

In [38]:
from pdfminer.high_level import extract_pages, extract_text
import pandas as pd
import re
import fitz 
import PIL.Image
import io
import os
import pdfplumber
import PyPDF2
from tabula.io import read_pdf
from tabulate import tabulate

In [39]:
# getting pdf files's names in a directory and ignoring other files.
def get_pdf_files():
    folder_path = 'crash and claims reports'
    files_names_ext = os.listdir(folder_path)
    pdf_files = [f for f in files_names_ext if f.endswith('.pdf')]
    global file_names
    file_names = [name.replace(".pdf", "") for name in pdf_files]

In [40]:
# creating a specific folder, if it does not exist
def folder_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [41]:
######### Extracting text #########
def extract__text():
    folder_exists('collected data/texts')
    for file in file_names:
        text = extract_text('crash and claims reports/'+file+'.pdf')
        with open(f"collected data/texts/{file}.txt", "w") as file:
            file.write(text)

In [42]:
######### Extracting Images #########
def extract__images():
    folder_exists('collected data/images')
    for file in file_names:
        pdf = fitz.open(f'crash and claims reports/{file}.pdf')
        os.mkdir(os.path.join('collected data/images/', file))
        counter = 1
        for i in range(len(pdf)):
            page = pdf[i]
            images = page.get_images()
            for image in images:
                base_img = pdf.extract_image(image[0])
                image_data = base_img["image"]
                img = PIL.Image.open(io.BytesIO(image_data))
                extension = base_img["ext"]
                img.save(open(f"collected data/images/{file}/image{counter}.{extension}", "wb"))
                counter += 1

In [43]:
######### Extracting Tables #########
def extract__tables():
    folder_exists('collected data/tables')
    
    for file in file_names:
        with pdfplumber.open(f'crash and claims reports/{file}.pdf') as pdf:
            # Create a new Excel workbook
            writer = pd.ExcelWriter(f'collected data/tables/{file}.xlsx')

            # Loop through each page in the PDF
            for i, page in enumerate(pdf.pages):
                # Extract the tables from the page
                tables = page.extract_tables()

                # Loop through each table in the page
                for j, table in enumerate(tables):
                    if len(table) > 10:
                        # Convert the table to a pandas DataFrame
                        df = pd.DataFrame(table)

                        # Set the sheet name in the workbook
                        sheet_name = f'Page {i+1}'
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
            writer.save()

In [59]:
######### Extracting Tables #########
def extract__tables2():
    folder_exists('collected data/tables')
    
    for file in file_names:
        pdf_reader = PyPDF2.PdfReader(f'crash and claims reports/{file}.pdf')
        pages = len(pdf_reader.pages)
        
        writer = pd.ExcelWriter(f'collected data/tables/{file}.xlsx')
        
        for i in range(pages):
            tables = read_pdf(f'crash and claims reports/{file}.pdf', pages= i+1, multiple_tables=True, 
                              stream=True, guess=False, pandas_options={'header': None})
            if(len(tables) == 0):
                continue
            
            for j,table in enumerate(tables):  
                sheet_name = f'Page {i+1} table{j+1}'
                table.to_excel(writer, sheet_name=sheet_name)
        writer.save()

In [9]:
# extract__text()

In [10]:
# extract__images()

In [60]:
get_pdf_files()
extract__tables2()

Got stderr: May 19, 2023 12:40:48 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 19, 2023 12:40:48 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 19, 2023 12:40:48 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 19, 2023 12:40:48 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

